# Step 1: Load Data

## Step 1.1: Import Libraries

In [1]:
from joblib import parallel_backend

import pandas as pd # Pandas library
import numpy as np # NumPy library

# Import Model Training Libaries
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None) # No max limit on number of columns displayed
pd.set_option('display.max_rows', None) # No max limit on number of rows displayed

In [3]:
pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


## Step 1.2: Load CSV

In [5]:
df = pd.read_csv('sales_train_evaluation.csv', index_col=False)

df.head(5)

id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  d_11  d_12  \
0       CA    0    0    0    0    0    0    0    0    0     0     0     0   
1       CA    0    0    0    0    0    0    0    0    0     0     0     0   
2       CA    0    0    0    0    0    0    0    0    0     0     0     0   
3       CA    0    0    0    0    0    0    0    0    0     0     0     0   
4       CA    0    0    0    0    0    0    0    0    0     0     0     0   

   d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  d_23  d_24  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  d_35  d_36  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  d_47  d_48  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     2     0     0     0     2     0     1     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_49  d_50  d_51  d_52  d_53  d_54  d_55  d_56  d_57  d_58  d_59  d_60  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     2     0     1     0     0     1     1     1     0     2     3   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_61  d_62  d_63  d_64  d_65  d_66  d_67  d_68  d_69  d_70  d_71  d_72  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     1     0     0     0     0     0     1     0     0     1     1     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_73  d_74  d_75  d_76  d_77  d_78  d_79  d_80  d_81  d_82  d_83  d_84  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     1     0     0     1     2     3     0     2     0     0     2   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_85  d_86  d_87  d_88  d_89  d_90  d_91  d_92  d_93  d_94  d_95  d_96  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   


## Step 1.3: Convert to Long Format

In [8]:
# Convert to long format

df.reset_index(inplace=True)  # item_id becomes a column again

id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
value_vars = [col for col in df.columns if col.startswith('d_')]

df_long = pd.melt(df, 
                    id_vars=id_vars, 
                    value_vars=value_vars, 
                    var_name='day', 
                    value_name='sales')

df_long.head(5)

id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  day  sales  
0       CA  d_1      0  
1       CA  d_1      0  
2       CA  d_1      0  
3       CA  d_1      0  
4       CA  d_1      0

In [9]:
# Extract the numeric part from 'day' (e.g., "d_1" → 1, "d_10" → 10)
df_long['day_num'] = df_long['day'].str.extract('(\d+)').astype(int)

# Sort by 'id' and 'day_num' instead of 'day'
df_long.sort_values(by=['id', 'day_num'], inplace=True)

# Drop the temporary column (optional)
df_long.drop('day_num', axis=1, inplace=True)

# Reset index
df_long.reset_index(drop=True, inplace=True)

df_long.head()

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/1478758091.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_long['day_num'] = df_long['day'].str.extract('(\d+)').astype(int)


id      item_id  dept_id cat_id store_id state_id  \
0  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
1  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
2  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
3  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
4  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   

   day  sales  
0  d_1      3  
1  d_2      0  
2  d_3      0  
3  d_4      1  
4  d_5      4

## Step 1.4: Sort by "id", "day"

In [11]:
# Create a numeric day column for correct sorting

df_long['day_num'] = df_long['day'].str.extract('d_(\d+)').astype(int)

# Sort using the numeric version
df_long.sort_values(by=['id', 'day_num'], inplace=True)
df_long.reset_index(drop=True, inplace=True)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/1712942370.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_long['day_num'] = df_long['day'].str.extract('d_(\d+)').astype(int)


In [12]:
df_long.head()

id      item_id  dept_id cat_id store_id state_id  \
0  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
1  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
2  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
3  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
4  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   

   day  sales  day_num  
0  d_1      3        1  
1  d_2      0        2  
2  d_3      0        3  
3  d_4      1        4  
4  d_5      4        5

# Step 2: Add Features

## ‼️ Checkpoint #1

In [15]:
df_features = df_long

## Step 2.1: 🗓️ Calendar Features

In [17]:
# Step 1: Load calendar data
calendar = pd.read_csv("calendar.csv")

# Step 2: Count number of events per day (vectorized)
calendar['events_per_day'] = (
    (calendar['event_type_1'] != 'no_event').astype(np.int8) +
    (calendar['event_type_2'] != 'no_event').astype(np.int8)
)

# Step 3: Merge calendar features into your main dataframe
df_features = df_long.merge(
    calendar[['d', 'wm_yr_wk', 'wday', 'month', 'year',
              'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
              'snap_CA', 'snap_TX', 'snap_WI', 'events_per_day']],
    left_on='day', right_on='d',
    how='left'
)

In [18]:
def reduce_mem_usage(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type in ['int64', 'int32']:
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif col_type in ['float64', 'float32']:
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif col_type == 'object':
            if df[col].nunique() < df.shape[0] // 10:
                df[col] = df[col].astype('category')
    return df

df_features = reduce_mem_usage(df_features)

In [19]:
df_features.head(10)

id      item_id  dept_id cat_id store_id state_id  \
0  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
1  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
2  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
3  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
4  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
5  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
6  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
7  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
8  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
9  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   

    day  sales  day_num     d  wm_yr_wk  wday  month  year event_name_1  \
0   d_1      3        1   d_1     11101     1      1  2011          NaN   
1   d_2      0        2   d_2     11101     2      1  2011          NaN   
2   d_3      0        3   d_3     11101     3      1  2011          NaN   
3   d_4      1        4   d_4     11101     4      2  2011          NaN   
4   d_5      4        5   d_5     11101     5      2  2011          NaN   
5   d_6      2        6   d_6     11101     6      2  2011          NaN   
6   d_7      0        7   d_7     11101     7      2  2011          NaN   
7   d_8      2        8   d_8     11102     1      2  2011          NaN   
8   d_9      0        9   d_9     11102     2      2  2011    SuperBowl   
9  d_10      0       10  d_10     11102     3      2  2011          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  \
0          NaN          NaN          NaN        0        0        0   
1          NaN          NaN          NaN        0        0        0   
2          NaN          NaN          NaN        0        0        0   
3          NaN          NaN          NaN        1        1        0   
4          NaN          NaN          NaN        1        0        1   
5          NaN          NaN          NaN        1        1        1   
6          NaN          NaN          NaN        1        0        0   
7          NaN          NaN          NaN        1        1        1   
8     Sporting          NaN          NaN        1        1        1   
9          NaN          NaN          NaN        1        1        0   

   events_per_day  
0               2  
1               2  
2               2  
3               2  
4               2  
5               2  
6               2  
7               2  
8               2  
9               2

## Step 2.2: 📦 Item-Store-Level Features

### Add Features: Lags and Rolling Means

#Lags

Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems.

Lag is expressed in a time unit & corresponds to the amount of data history we allow the model to use when making the prediction.

Here we have applied Lags on 'demand' column.

The maximum Lags taken is 70 days

In [24]:
# Generate lag_days for 1, 7, 14, 28, 35, 42, 49, 56, 63, 70 days
from tqdm import tqdm

# List of desired lag periods
lag_days = [1, 7, 14, 28, 35, 42, 49, 56, 63, 70]

# Generate lag features for 'sales' grouped by 'id'
for lag in tqdm(lag_days, desc = 'Generating lag features', disable=True):
    df_features[f'lag_{lag}'] = df_features.groupby('id')['sales'].shift(lag)

/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/3612538011.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_features[f'lag_{lag}'] = df_features.groupby('id')['sales'].shift(lag)
/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/3612538011.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_features[f'lag_{lag}'] = df_features.groupby('id')['sales'].shift(lag)
/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/3612538011.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=F

In [25]:
for lag in lag_days:
    col = f'lag_{lag}'
    df_features[col] = df_features[col].fillna(0)

# Display the first 5 rows
df_features.head()

id      item_id  dept_id cat_id store_id state_id  \
0  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
1  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
2  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
3  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
4  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   

   day  sales  day_num    d  wm_yr_wk  wday  month  year event_name_1  \
0  d_1      3        1  d_1     11101     1      1  2011          NaN   
1  d_2      0        2  d_2     11101     2      1  2011          NaN   
2  d_3      0        3  d_3     11101     3      1  2011          NaN   
3  d_4      1        4  d_4     11101     4      2  2011          NaN   
4  d_5      4        5  d_5     11101     5      2  2011          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  \
0          NaN          NaN          NaN        0        0        0   
1          NaN          NaN          NaN        0        0        0   
2          NaN          NaN          NaN        0        0        0   
3          NaN          NaN          NaN        1        1        0   
4          NaN          NaN          NaN        1        0        1   

   events_per_day  lag_1  lag_7  lag_14  lag_28  lag_35  lag_42  lag_49  \
0               2    0.0    0.0     0.0     0.0     0.0     0.0     0.0   
1               2    3.0    0.0     0.0     0.0     0.0     0.0     0.0   
2               2    0.0    0.0     0.0     0.0     0.0     0.0     0.0   
3               2    0.0    0.0     0.0     0.0     0.0     0.0     0.0   
4               2    1.0    0.0     0.0     0.0     0.0     0.0     0.0   

   lag_56  lag_63  lag_70  
0     0.0     0.0     0.0  
1     0.0     0.0     0.0  
2     0.0     0.0     0.0  
3     0.0     0.0     0.0  
4     0.0     0.0     0.0

#Rolling Mean
* Rolling is a very useful operation for time series data.

* Rolling mean helps smooth out short-term fluctuations and highlight longer-term trends or cycles
  
* Rolling means creating a rolling window with a specified size & perform calculations on data in this window which rolls through data

* Here we have Rolling-Mean on 'demand' column and the max Window size taken is 42

In [27]:
from tqdm import tqdm

# Define rolling window sizes
rolling_windows = [7, 14, 28, 35, 42]

# Generate rolling mean features for 'sales', shifted to avoid leakage
for window in tqdm(rolling_windows, desc="Generating rolling mean features", disable=True):
    col_name = f'rolling_mean_{window}'
    df_features[col_name] = (
        df_features
        .groupby('id')['sales']
        .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).mean())
    )
    df_features[col_name] = df_features[col_name].fillna(0)


/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/3682397902.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('id')['sales']
/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/3682397902.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('id')['sales']
/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/3682397902.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this war

In [28]:
df_features.head(10)

id      item_id  dept_id cat_id store_id state_id  \
0  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
1  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
2  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
3  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
4  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
5  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
6  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
7  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
8  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
9  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   

    day  sales  day_num     d  wm_yr_wk  wday  month  year event_name_1  \
0   d_1      3        1   d_1     11101     1      1  2011          NaN   
1   d_2      0        2   d_2     11101     2      1  2011          NaN   
2   d_3      0        3   d_3     11101     3      1  2011          NaN   
3   d_4      1        4   d_4     11101     4      2  2011          NaN   
4   d_5      4        5   d_5     11101     5      2  2011          NaN   
5   d_6      2        6   d_6     11101     6      2  2011          NaN   
6   d_7      0        7   d_7     11101     7      2  2011          NaN   
7   d_8      2        8   d_8     11102     1      2  2011          NaN   
8   d_9      0        9   d_9     11102     2      2  2011    SuperBowl   
9  d_10      0       10  d_10     11102     3      2  2011          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  \
0          NaN          NaN          NaN        0        0        0   
1          NaN          NaN          NaN        0        0        0   
2          NaN          NaN          NaN        0        0        0   
3          NaN          NaN          NaN        1        1        0   
4          NaN          NaN          NaN        1        0        1   
5          NaN          NaN          NaN        1        1        1   
6          NaN          NaN          NaN        1        0        0   
7          NaN          NaN          NaN        1        1        1   
8     Sporting          NaN          NaN        1        1        1   
9          NaN          NaN          NaN        1        1        0   

   events_per_day  lag_1  lag_7  lag_14  lag_28  lag_35  lag_42  lag_49  \
0               2    0.0    0.0     0.0     0.0     0.0     0.0     0.0   
1               2    3.0    0.0     0.0     0.0     0.0     0.0     0.0   
2               2    0.0    0.0     0.0     0.0     0.0     0.0     0.0   
3               2    0.0    0.0     0.0     0.0     0.0     0.0     0.0   
4               2    1.0    0.0     0.0     0.0     0.0     0.0     0.0   
5               2    4.0    0.0     0.0     0.0     0.0     0.0     0.0   
6               2    2.0    0.0     0.0     0.0     0.0     0.0     0.0   
7               2    0.0    3.0     0.0     0.0     0.0     0.0     0.0   
8               2    2.0    0.0     0.0     0.0     0.0     0.0     0.0   
9               2    0.0    0.0     0.0     0.0     0.0     0.0     0.0   

   lag_56  lag_63  lag_70  rolling_mean_7  rolling_mean_14  rolling_mean_28  \
0     0.0     0.0     0.0        0.000000         0.000000         0.000000   
1     0.0     0.0     0.0        3.000000         3.000000         3.000000   
2     0.0     0.0     0.0        1.500000         1.500000         1.500000   
3     0.0     0.0     0.0        1.000000         1.000000         1.000000   
4     0.0     0.0     0.0        1.000000         1.000000         1.000000   
5     0.0     0.0     0.0        1.600000         1.600000         1.600000   
6     0.0     0.0     0.0        1.666667         1.666667         1.666667   
7     0.0     0.0     0.0        1.428571         1.428571         1.4

## ‼️ Checkpoint #2

## Step 2.3: 🏬 Store-Level Features

In [31]:
df_aggregated = df_features

### Step 2.3.1: store_avg_item_sales_ratio

In [33]:
import gc  # For manual garbage collection
# Step 1: Compute total store sales per day
df_store_sales = df_aggregated.groupby(['store_id', 'day'], as_index=False)['sales'].sum()
df_store_sales.rename(columns={'sales': 'store_total_sales'}, inplace=True)

# Step 2: Merge back into main DataFrame
df_aggregated = df_aggregated.merge(df_store_sales, on=['store_id', 'day'], how='left')

# Step 3: Compute item-to-store sales ratio
df_aggregated['store_avg_item_sales_ratio'] = df_aggregated['sales'] / (df_aggregated['store_total_sales'] + 1e-6)

# Step 4: Clean up
df_aggregated.drop(columns=['store_total_sales'], inplace=True)
del df_store_sales
gc.collect()


/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/1092899713.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_store_sales = df_aggregated.groupby(['store_id', 'day'], as_index=False)['sales'].sum()


14

In [34]:
df_aggregated = df_aggregated.drop(columns = ['d'])
df_aggregated.head()

id      item_id  dept_id cat_id store_id state_id  \
0  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
1  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
2  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
3  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   
4  FOODS_1_001_CA_1_evaluation  FOODS_1_001  FOODS_1  FOODS     CA_1       CA   

   day  sales  day_num  wm_yr_wk  wday  month  year event_name_1 event_type_1  \
0  d_1      3        1     11101     1      1  2011          NaN          NaN   
1  d_2      0        2     11101     2      1  2011          NaN          NaN   
2  d_3      0        3     11101     3      1  2011          NaN          NaN   
3  d_4      1        4     11101     4      2  2011          NaN          NaN   
4  d_5      4        5     11101     5      2  2011          NaN          NaN   

  event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  events_per_day  lag_1  \
0          NaN          NaN        0        0        0               2    0.0   
1          NaN          NaN        0        0        0               2    3.0   
2          NaN          NaN        0        0        0               2    0.0   
3          NaN          NaN        1        1        0               2    0.0   
4          NaN          NaN        1        0        1               2    1.0   

   lag_7  lag_14  lag_28  lag_35  lag_42  lag_49  lag_56  lag_63  lag_70  \
0    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   rolling_mean_7  rolling_mean_14  rolling_mean_28  rolling_mean_35  \
0             0.0              0.0              0.0              0.0   
1             3.0              3.0              3.0              3.0   
2             1.5              1.5              1.5              1.5   
3             1.0              1.0              1.0              1.0   
4             1.0              1.0              1.0              1.0   

   rolling_mean_42  store_avg_item_sales_ratio  
0              0.0                    0.000692  
1              3.0                    0.000000  
2              1.5                    0.000000  
3              1.0                    0.000328  
4              1.0                    0.001521

## ‼️ Checkpoint #3

## Step 2.4: 🗃️ Category / Department + Location Features

In [37]:
df_aggregated_2 = df_aggregated

Category × Store rolling mean for 14 and 28 days

In [40]:
import gc
from tqdm import tqdm

windows = [14, 28]
unique_combos = df_aggregated_2[['cat_id', 'store_id']].drop_duplicates()

for w in windows:
    rolling_col = f'cat_store_rolling_mean_{w}d'
    df_aggregated_2[rolling_col] = np.nan  # Preallocate to avoid column growth

    for _, row in tqdm(unique_combos.iterrows(), total=len(unique_combos),
                       desc=f'Rolling {w}d: cat-store chunks', disable=True):
        cat_id = row['cat_id']
        store_id = row['store_id']

        # Slice only the chunk for one category-store pair
        mask = (df_aggregated_2['cat_id'] == cat_id) & (df_aggregated_2['store_id'] == store_id)
        df_chunk = df_aggregated_2.loc[mask, ['day', 'sales']].copy()

        # Sort by day to ensure correct rolling
        df_chunk.sort_values('day', inplace=True)

        # Compute rolling mean safely
        rolled = (
            df_chunk['sales']
            .shift(1)
            .rolling(window=w, min_periods=1)
            .mean()
            .values
        )

        # Write back result into main df_features
        df_aggregated_2.loc[mask, rolling_col] = rolled

    gc.collect()  # Free memory at the end of each window


Department × State rolling mean for 14 and 28 days

In [42]:
import gc
from tqdm import tqdm

windows = [14, 28]
unique_combos = df_features[['dept_id', 'state_id']].drop_duplicates()

for w in windows:
    rolling_col = f'dept_state_rolling_mean_{w}d'
    df_aggregated_2[rolling_col] = np.nan  # Pre-allocate the column

    for _, row in tqdm(unique_combos.iterrows(), total=len(unique_combos),
                       desc=f'Rolling {w}d: dept-state chunks', disable=True):
        dept_id = row['dept_id']
        state_id = row['state_id']

        # Slice chunk
        mask = (df_aggregated_2['dept_id'] == dept_id) & (df_aggregated_2['state_id'] == state_id)
        df_chunk = df_aggregated_2.loc[mask, ['day', 'sales']].copy()
        df_chunk.sort_values('day', inplace=True)

        # Safe rolling mean (no leakage)
        rolled = (
            df_chunk['sales']
            .shift(1)
            .rolling(window=w, min_periods=1)
            .mean()
            .values
        )

        # Assign back to main DataFrame
        df_aggregated_2.loc[mask, rolling_col] = rolled

    gc.collect()  # Free memory at end of each rolling window


## ‼️ Checkpoint #4

## Step 2.5: 🌎 State / Country-Level Features

In [45]:
df_aggregated_3 = df_aggregated_2

### Step 2.5.1: state_cat_prev_week_sales + state_cat_weekly_growth

In [47]:
import numpy as np
import gc

# Step 1: Aggregate daily sales at state-category level
state_cat_sales = (
    df_aggregated_3
    .groupby(['state_id', 'cat_id', 'day'], as_index=False)['sales']
    .sum()
)

# Step 2: Compute 7-day rolling sum up to *yesterday* (no leakage)
state_cat_sales['weekly_sales'] = (
    state_cat_sales
    .groupby(['state_id', 'cat_id'])['sales']
    .transform(lambda x: x.shift(1).rolling(window=7, min_periods=1).sum())
).astype(np.float32)

# Step 3: Get previous week's sales (7-day window ending 8–14 days ago)
state_cat_sales['prev_week_sales'] = (
    state_cat_sales
    .groupby(['state_id', 'cat_id'])['weekly_sales']
    .shift(7)
).astype(np.float32)

# Step 4: Compute weekly growth safely
eps = 1e-6
state_cat_sales['weekly_growth'] = (
    (state_cat_sales['weekly_sales'] - state_cat_sales['prev_week_sales']) /
    (state_cat_sales['prev_week_sales'] + eps)
).astype(np.float32)

# Step 5: Merge features into df_features
df_aggregated_3 = df_aggregated_3.merge(
    state_cat_sales[['state_id', 'cat_id', 'day', 'prev_week_sales', 'weekly_growth']],
    on=['state_id', 'cat_id', 'day'],
    how='left'
)

# Step 6: Rename columns and cleanup
df_aggregated_3.rename(columns={
    'prev_week_sales': 'state_cat_prev_week_sales',
    'weekly_growth': 'state_cat_weekly_growth'
}, inplace=True)

# Optional: Clear memory
del state_cat_sales
gc.collect()


/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/2487817237.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['state_id', 'cat_id', 'day'], as_index=False)['sales']
/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/2487817237.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['state_id', 'cat_id'])['sales']
/var/folders/b3/td5c1lpn10n290bskyjhkwkh0000gn/T/ipykernel_1840/2487817237.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obse

0

## ‼️ Checkpoint #5

In [49]:
df_aggregated_4 = df_aggregated_3

## Step 2.6: 💲 Prices

In [ ]:
import pandas as pd
import numpy as np
import gc

# === Step 0: Load sell_prices with efficient dtypes ===
dtype_dict = {
    'store_id': 'category',
    'item_id': 'category',
    'wm_yr_wk': 'int32',  # Load first as int
    'sell_price': 'float32'
}

sell_prices = pd.read_csv(
    "sell_prices.csv",
    usecols=['store_id', 'item_id', 'wm_yr_wk', 'sell_price'],
    dtype=dtype_dict
)

# Convert to category after load
sell_prices['wm_yr_wk'] = sell_prices['wm_yr_wk'].astype('category')

# === Step 1: Match df_aggregated_4 dtypes and align categories ===
for col in ['store_id', 'item_id', 'wm_yr_wk']:
    df_aggregated_4[col] = df_aggregated_4[col].astype('category')
    sell_prices[col] = sell_prices[col].astype('category')

    # Use union of categories to ensure no category is dropped
    all_cats = df_aggregated_4[col].cat.categories.union(sell_prices[col].cat.categories)
    df_aggregated_4[col] = df_aggregated_4[col].cat.set_categories(all_cats)
    sell_prices[col] = sell_prices[col].cat.set_categories(all_cats)

# === Step 2: Drop old sell_price column (if already exists) ===
if 'sell_price' in df_aggregated_4.columns:
    df_aggregated_4.drop(columns='sell_price', inplace=True)

# === Step 3: Left join to retain all df_aggregated_4 rows ===
df_aggregated_4 = pd.merge(
    df_aggregated_4,
    sell_prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left',
    sort=False,
    copy=False
)

# === Step 4: Validate ===
print("Unique store_id count after merge:", df_aggregated_4['store_id'].nunique())  # Should be 10

# === Step 5: Clean up ===
del sell_prices
gc.collect()


In [ ]:
df_aggregated_4.columns

In [ ]:
df_aggregated_4[['id','item_id','dept_id','cat_id','store_id','state_id']].nunique()

In [ ]:
df_aggregated_4['store_id'].unique()

## ‼️ Checkpoint #5

# Step 3: Reduce DataFrame Size

## Step 3.1: Remove Unnecessary Data

In [ ]:
del df_aggregated
del df_aggregated_2
del df_aggregated_3

In [ ]:
import gc

gc.collect()

## Step 3.2: Convert Columns' Data Type

In [71]:
df_aggregated_5 = df_aggregated_4

In [73]:
# Check data types before optimization
df_aggregated_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 43 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   id                           category
 1   item_id                      category
 2   dept_id                      category
 3   cat_id                       category
 4   store_id                     category
 5   state_id                     category
 6   day                          category
 7   sales                        int16   
 8   day_num                      int16   
 9   wm_yr_wk                     int16   
 10  wday                         int8    
 11  month                        int8    
 12  year                         int16   
 13  event_name_1                 category
 14  event_type_1                 category
 15  event_name_2                 category
 16  event_type_2                 category
 17  snap_CA                      int8    
 18  snap_TX             

In [75]:
# # Convert object columns to 'category' if they have a small number of unique values

# df_aggregated_5['id'] = df_aggregated_5['id'].astype('category')
# df_aggregated_5['item_id'] = df_aggregated_5['item_id'].astype('category')
# df_aggregated_5['dept_id'] = df_aggregated_5['dept_id'].astype('category')
# df_aggregated_5['cat_id'] = df_aggregated_5['cat_id'].astype('category')
# df_aggregated_5['store_id'] = df_aggregated_5['store_id'].astype('category')
# df_aggregated_5['state_id'] = df_aggregated_5['state_id'].astype('category')
# df_aggregated_5['day'] = df_aggregated_5['day'].astype('category')
# # df_aggregated_5['d'] = df_aggregated_5['d'].astype('category')
# # df_aggregated_5['weekday'] = df_aggregated_5['weekday'].astype('category')
# df_aggregated_5['event_name_1'] = df_aggregated_5['event_name_1'].astype('category')
# df_aggregated_5['event_type_1'] = df_aggregated_5['event_type_1'].astype('category')

# # Convert int columns to smaller data types
# df_aggregated_5['day_num'] = df_aggregated_5['day_num'].astype('int32')
# df_aggregated_5['wm_yr_wk'] = df_aggregated_5['wm_yr_wk'].astype('int32')
# df_aggregated_5['wday'] = df_aggregated_5['wday'].astype('int8')
# df_aggregated_5['month'] = df_aggregated_5['month'].astype('int8')
# df_aggregated_5['year'] = df_aggregated_5['year'].astype('int16')
# df_aggregated_5['snap_CA'] = df_aggregated_5['snap_CA'].astype('int8')
# df_aggregated_5['snap_TX'] = df_aggregated_5['snap_TX'].astype('int8')
# df_aggregated_5['snap_WI'] = df_aggregated_5['snap_WI'].astype('int8')
# df_aggregated_5['sales_store_total'] = df_aggregated_5['sales_store_total'].astype('int32')
# df_aggregated_5['country_sales'] = df_aggregated_5['country_sales'].astype('int32')

# # Convert float columns to smaller data types
# df_aggregated_5['item_lag_1d'] = df_aggregated_5['item_lag_1d'].astype('float32')
# df_aggregated_5['item_rolling_mean_7d'] = df_aggregated_5['item_rolling_mean_7d'].astype('float32')
# df_aggregated_5['item_rolling_std_30d'] = df_aggregated_5['item_rolling_std_30d'].astype('float32')
# df_aggregated_5['store_total_sales_7d'] = df_aggregated_5['store_total_sales_7d'].astype('float32')
# df_aggregated_5['store_avg_item_sales_ratio'] = df_aggregated_5['store_avg_item_sales_ratio'].astype('float32')
# df_aggregated_5['cat_store_rolling_mean_14d'] = df_aggregated_5['cat_store_rolling_mean_14d'].astype('float32')
# df_aggregated_5['dept_state_rolling_mean_30d'] = df_aggregated_5['dept_state_rolling_mean_30d'].astype('float32')
# df_aggregated_5['state_cat_prev_week_sales'] = df_aggregated_5['state_cat_prev_week_sales'].astype('float32')
# df_aggregated_5['state_cat_weekly_growth'] = df_aggregated_5['state_cat_weekly_growth'].astype('float32')
# df_aggregated_5['sell_price'] = df_aggregated_5['sell_price'].astype('float32')

In [77]:
from sklearn.preprocessing import LabelEncoder

# === Step 1: Downcast numeric columns ===
float64_cols = df_aggregated_5.select_dtypes(include='float64').columns
int64_cols = df_aggregated_5.select_dtypes(include='int64').columns

for col in float64_cols:
    df_aggregated_5[col] = pd.to_numeric(df_aggregated_5[col], downcast='float')

for col in int64_cols:
    df_aggregated_5[col] = pd.to_numeric(df_aggregated_5[col], downcast='integer')

# === Step 2: Cast object/low-cardinality columns to category ===
category_cols = [
    'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
    'day', 'wm_yr_wk', 'event_name_1', 'event_type_1',
    'event_name_2', 'event_type_2'
]

for col in category_cols:
    df_aggregated_5[col] = df_aggregated_5[col].astype('category')

# === Step 3: Downcast small-range ints manually (if still not optimal) ===
manual_downcast = {
    'wday': 'int8',
    'month': 'int8',
    'year': 'int16',
    'snap_CA': 'int8',
    'snap_TX': 'int8',
    'snap_WI': 'int8',
    'sales': 'int16',
    'day_num': 'int16',
}

for col, dtype in manual_downcast.items():
    df_aggregated_5[col] = df_aggregated_5[col].astype(dtype)

# === Step 4: Print new memory usage ===
print(df_aggregated_5.info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 43 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   id                           category
 1   item_id                      category
 2   dept_id                      category
 3   cat_id                       category
 4   store_id                     category
 5   state_id                     category
 6   day                          category
 7   sales                        int16   
 8   day_num                      int16   
 9   wm_yr_wk                     category
 10  wday                         int8    
 11  month                        int8    
 12  year                         int16   
 13  event_name_1                 category
 14  event_type_1                 category
 15  event_name_2                 category
 16  event_type_2                 category
 17  snap_CA                      int8    
 18  snap_TX             

## Label-Encoding

####  Label Encoding → Work better for Sequential (Deep Learning) Models

#### Category Dtype → Work better for Tree-Based Models

Encoding refers to converting the labels into numeric form so as to convert it into the machine-readable form.

Machine learning algorithms can then decide in a better way on how those labels must be operated.

It is an important pre-processing step for the structured dataset in supervised learning

Label-encoding: 

In [79]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# List of categorical columns to encode
cat_cols = [
    'event_name_1', 'event_type_1',
    'event_name_2', 'event_type_2',
    'id', 'item_id', 'dept_id',
    'cat_id', 'store_id', 'state_id'
]

# Encode each category column and add a new column with trailing underscore
labelencoder = LabelEncoder()

for col in tqdm(cat_cols, desc="Label encoding categorical features", disable=True):
    df_aggregated_5[col + '_'] = labelencoder.fit_transform(df_aggregated_5[col].astype(str))


In [80]:
# Check the memory usage after optimization

df_aggregated_5.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 53 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   id                           category
 1   item_id                      category
 2   dept_id                      category
 3   cat_id                       category
 4   store_id                     category
 5   state_id                     category
 6   day                          category
 7   sales                        int16   
 8   day_num                      int16   
 9   wm_yr_wk                     category
 10  wday                         int8    
 11  month                        int8    
 12  year                         int16   
 13  event_name_1                 category
 14  event_type_1                 category
 15  event_name_2                 category
 16  event_type_2                 category
 17  snap_CA                      int8    
 18  snap_TX             

## ‼️ Checkpoint #6

In [87]:
df_aggregated_final = df_aggregated_5

In [91]:
df_aggregated_final.tail()

id          item_id      dept_id  \
59181085  HOUSEHOLD_2_516_WI_3_evaluation  HOUSEHOLD_2_516  HOUSEHOLD_2   
59181086  HOUSEHOLD_2_516_WI_3_evaluation  HOUSEHOLD_2_516  HOUSEHOLD_2   
59181087  HOUSEHOLD_2_516_WI_3_evaluation  HOUSEHOLD_2_516  HOUSEHOLD_2   
59181088  HOUSEHOLD_2_516_WI_3_evaluation  HOUSEHOLD_2_516  HOUSEHOLD_2   
59181089  HOUSEHOLD_2_516_WI_3_evaluation  HOUSEHOLD_2_516  HOUSEHOLD_2   

             cat_id store_id state_id     day  sales  day_num wm_yr_wk  wday  \
59181085  HOUSEHOLD     WI_3       WI  d_1937      0     1937    11616     5   
59181086  HOUSEHOLD     WI_3       WI  d_1938      0     1938    11616     6   
59181087  HOUSEHOLD     WI_3       WI  d_1939      0     1939    11616     7   
59181088  HOUSEHOLD     WI_3       WI  d_1940      0     1940    11617     1   
59181089  HOUSEHOLD     WI_3       WI  d_1941      0     1941    11617     2   

          month  year event_name_1 event_type_1 event_name_2 event_type_2  \
59181085      5  2016          NaN          NaN          NaN          NaN   
59181086      5  2016          NaN          NaN          NaN          NaN   
59181087      5  2016          NaN          NaN          NaN          NaN   
59181088      5  2016          NaN          NaN          NaN          NaN   
59181089      5  2016          NaN          NaN          NaN          NaN   

          snap_CA  snap_TX  snap_WI  events_per_day  lag_1  lag_7  lag_14  \
59181085        0        0        0               2    0.0    0.0     0.0   
59181086        0        0        0               2    0.0    0.0     0.0   
59181087        0        0        0               2    0.0    0.0     0.0   
59181088        0        0        0               2    0.0    1.0     0.0   
59181089        0        0        0               2    0.0    0.0     0.0   

          lag_28  lag_35  lag_42  lag_49  lag_56  lag_63  lag_70  \
59181085     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
59181086     0.0     0.0     0.0     0.0     0.0     0.0     1.0   
59181087     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
59181088     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
59181089     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

          rolling_mean_7  rolling_mean_14  rolling_mean_28  rolling_mean_35  \
59181085        0.142857         0.071429         0.071429         0.057143   
59181086        0.142857         0.071429         0.071429         0.057143   
59181087        0.142857         0.071429         0.071429         0.057143   
59181088        0.142857         0.071429         0.071429         0.057143   
59181089        0.000000         0.071429         0.071429         0.057143   

          rolling_mean_42  store_avg_item_sales_ratio  \
59181085         0.047619                         0.0   
59181086         0.047619                         0.0   
59181087         0.047619                         0.0   
59181088         0.047619                         0.0   
59181089         0.047619                         0.0   

          cat_store_rolling_mean_14d  cat_store_rolling_mean_28d  \
59181085                    0.500000                    0.321429   
59181086                    0.357143                    0.321429   
59181087                    0.428571                    0.357143   
59181088                    0.428571                    0.357143   
59181089                    0.428571                    0.357143   

          dept_state_rolling_mean_14d  dept_state_rolling_mean_28d  \
59181085                     0.285714                     0.142857   
59181086                     0.285714                     0.142857   
59181087                     0.285714                     0.142857   
59181088                     0.285714                     0.142857   
59181089                     0.285714                     0.142857   

          state_cat_prev_week_sales  state_cat_weekly_growth  event_name_1_  \
59181085                    16516.0          

## Step 3.3: Preliminary Export to Parquet

In [93]:
df_aggregated_final.to_parquet('Long_DataFrame_Sirui_V2.parquet', index=False, engine='fastparquet')

# Step 4: Organize Data for Pipeline

## Step 4.1: Remove Unwanted Columns

In [123]:
# df_aggregated_final = df_aggregated_final.drop(columns=['day_num', 'wday'])

## Step 4.2: Move Target Variable 'sales' to Last Column

In [ ]:
# sales_column = df_aggregated_final.pop('sales')
# df_aggregated_final['sales'] = sales_column

## Step 4.3: Rename Columns for Clarity

In [ ]:
# Rename columns one by one to minimize memory impact

In [ ]:
# df_aggregated_final['store_item_lag_1d'] = df_aggregated_final.pop('item_lag_1d')

In [ ]:
# df_aggregated_final['store_item_rolling_mean_7d'] = df_aggregated_final.pop('item_rolling_mean_7d')

In [ ]:
# df_aggregated_final['store_item_rolling_std_30d'] = df_aggregated_final.pop('item_rolling_std_30d')

In [ ]:
# df_aggregated_final['store_total_sales'] = df_aggregated_final.pop('sales_store_total')

In [ ]:
# df_aggregated_final['store_cat_rolling_mean_14d'] = df_aggregated_final.pop('cat_store_rolling_mean_14d')

In [ ]:
# df_aggregated_final['state_dept_rolling_mean_30d'] = df_aggregated_final.pop('dept_state_rolling_mean_30d')

In [ ]:
# df_aggregated_final['state_item_week_sell_price'] = df_aggregated_final.pop('sell_price')

In [ ]:
# # Check the result

# df_aggregated_final.head(25)

# Step 5: Export New Pandas DataFrame to External File

In [156]:
# Export to Parquet file
df_aggregated_final.to_parquet('Long_DataFrame_Sirui_V1.parquet', index=False, engine='fastparquet')

In [ ]:
# Export to CSV file
df_aggregated_final.to_csv('Long_DataFrame_Sirui_V1.csv', index=False, sep=';', compression='gzip')